# Module

In [ ]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import numpy as np

# Dataset

In [ ]:
# Mnist Data down 받기

# 공개 데이터셋
train_data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
# dataloader 생성
train_dataloader = DataLoader(train_data,batch_size=128)
test_dataloader = DataLoader(test_data,batch_size=32)

for x,y in test_dataloader:
    print('Shape of x {N,C,H,W}',x.shape)
    print('Shape of y:',y.shape,y.dtype)
    break

Shape of x {N,C,H,W} torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


In [ ]:
import torch
# 학습에 사용할 CPU나 GPU 장치
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

# Model 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(28*28,512)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(512,10)
    
    def forward(self,x):
        x = self.flatten(x)
        h1 = self.linear1(x)
        h1 = self.relu(h1) 
        output = self.linear2(h1)
        return output

Using cuda device


In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=784, out_features=512, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
# Loss 함수와 Optimizer 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(),lr=0.001)

In [ ]:
# Training을 위한 함수
def train(dataloader,model,loss_fn,optimizer):
    size = len(dataloader.dataset)
    for batch,(X,y) in enumerate(dataloader):
        X,y = X.to(device),y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred,y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 50 == 0:
            loss,current = loss.item(),batch*len(x)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')

In [ ]:
# Test를 위한 함수
def test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss,correct = 0,0
    with torch.no_grad():
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model(X)
            test_loss = loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error : \n Accuracy: {(100*correct):>0.1f}%, Avg loss : {test_loss:>0f} \n')

In [ ]:
epochs = 10
for t in range(epochs):
    print(f'Epoch {t+1}\n-----------------------')
    train(train_dataloader,model,loss_fn,optimizer)
    test(test_dataloader,model,loss_fn)
print('Done!')

Epoch 1
-----------------------
loss: 0.043411 [    0/60000]
loss: 0.025361 [ 1600/60000]
loss: 0.034329 [ 3200/60000]
loss: 0.055025 [ 4800/60000]
loss: 0.048036 [ 6400/60000]
loss: 0.098170 [ 8000/60000]
loss: 0.067083 [ 9600/60000]
loss: 0.077567 [11200/60000]
loss: 0.094423 [12800/60000]
loss: 0.037160 [14400/60000]
Test Error : 
 Accuracy: 97.5%, Avg loss : 0.000014 

Epoch 2
-----------------------
loss: 0.033435 [    0/60000]
loss: 0.012880 [ 1600/60000]
loss: 0.026733 [ 3200/60000]
loss: 0.031363 [ 4800/60000]
loss: 0.032162 [ 6400/60000]
loss: 0.049621 [ 8000/60000]
loss: 0.044461 [ 9600/60000]
loss: 0.048517 [11200/60000]
loss: 0.081028 [12800/60000]
loss: 0.018578 [14400/60000]
Test Error : 
 Accuracy: 97.6%, Avg loss : 0.000011 

Epoch 3
-----------------------
loss: 0.029511 [    0/60000]
loss: 0.008270 [ 1600/60000]
loss: 0.024498 [ 3200/60000]
loss: 0.027155 [ 4800/60000]
loss: 0.020574 [ 6400/60000]
loss: 0.032933 [ 8000/60000]
loss: 0.028198 [ 9600/60000]
loss: 0.02516